Implementation of Deep Net with 1 hidden layer

In [1]:
pip install tenseal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 6.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import tenseal as ts
import pandas as pd
import random
from time import time

# those are optional and are not necessary for training
import numpy as np
import matplotlib.pyplot as plt

In [3]:
torch.random.manual_seed(73)
random.seed(73)


def split_train_test(x, y, test_ratio=0.3):
    idxs = [i for i in range(len(x))]
    random.shuffle(idxs)
    # delimiter between test and train data
    delim = int(len(x) * test_ratio)
    test_idxs, train_idxs = idxs[:delim], idxs[delim:]
    return x[train_idxs], y[train_idxs], x[test_idxs], y[test_idxs]


def heart_disease_data():
    data = pd.read_csv("/kaggle/input/trydataset1/framingham.csv")
    # drop rows with missing values
    data = data.dropna()
    # drop some features
    data = data.drop(columns=["education", "currentSmoker", "BPMeds", "diabetes", "diaBP", "BMI"])
    # balance data
    grouped = data.groupby('TenYearCHD')
    data = grouped.apply(lambda x: x.sample(grouped.size().min(), random_state=73).reset_index(drop=True))
    # extract labels
    y = torch.tensor(data["TenYearCHD"].values).float().unsqueeze(1)
    #data = data.drop("TenYearCHD",'columns')
    # standardize data
    data = (data - data.mean()) / data.std()
    x = torch.tensor(data.values).float()
    return split_train_test(x, y)


def random_data(m=1024, n=2):
    # data separable by the line `y = x`
    x_train = torch.randn(m, n)
    x_test = torch.randn(m // 2, n)
    y_train = (x_train[:, 0] >= x_train[:, 1]).float().unsqueeze(0).t()
    y_test = (x_test[:, 0] >= x_test[:, 1]).float().unsqueeze(0).t()
    return x_train, y_train, x_test, y_test


# You can use whatever data you want without modification to the tutorial
# x_train, y_train, x_test, y_test = random_data()
x_train, y_train, x_test, y_test = heart_disease_data()

print("############# Data summary #############")
print(f"x_train has shape: {x_train.shape}")
print(f"y_train has shape: {y_train.shape}")
print(f"x_test has shape: {x_test.shape}")
print(f"y_test has shape: {y_test.shape}")
print("#######################################")

############# Data summary #############
x_train has shape: torch.Size([780, 10])
y_train has shape: torch.Size([780, 1])
x_test has shape: torch.Size([334, 10])
y_test has shape: torch.Size([334, 1])
#######################################


<ipython-input-3-9c1daa9f64ab>:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = grouped.apply(lambda x: x.sample(grouped.size().min(), random_state=73).reset_index(drop=True))


In [4]:
# parameters
poly_mod_degree = 8192
coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 21, 21, 40]
# create TenSEALContext
ctx_training = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
ctx_training.global_scale = 2 ** 21
ctx_training.generate_galois_keys()

In [5]:
#t_start = time()
#enc_x_train = ts.ckks_tensor(ctx_training, x_train[0:300])
#enc_y_train = ts.ckks_tensor(ctx_training, y_train[0:300])
#t_end = time()
#print(f"Encryption of the training_set took {int(t_end - t_start)} seconds")

In [6]:
class EncryptedDL1layer() :
  def __init__(self) -> None:
      self.weight1=np.random.rand(10,3)*0.01
      self.bias1=np.random.rand(3)*0.01
      self.weight2=np.random.rand(3, 1)*0.01
      self.bias2=np.random.rand(1)*0.01
      self.dw1=0
      self.db1=0
      self.dw2=0
      self.db2=0
  def bootstrapping(enc,ctx_training) :
    return ts.ckks_tensor(ctx_training,enc.decrypt())
  def sigmoid(enc_x):
    return enc_x.polyval([0, 1, 0, -0.3333,0,0.13333])
  def sigmoid_derv(enc_x):
    return enc_x.polyval([1,0,-0.6666,0.66665])
  def forward(self,enc_x_train,ctx_training) :
    z11=enc_x_train.mm(self.weight1)
    z1=z11.add(self.bias1)
    a1=EncryptedDL1layer.sigmoid(z1)
    y=EncryptedDL1layer.bootstrapping(a1,ctx_training)
    z21=y.mm(self.weight2)
    z2=z21.add(self.bias2)
    a2=EncryptedDL1layer.sigmoid(z2)
    return a2,z2,a1,z1
  def backward(self,a2,z2,a1,z1,enc_y_train,ctx_training) :
    #calculating the output at the layer 2
    error=a2-enc_y_train
    der=EncryptedDL1layer.sigmoid_derv(z2)
    #finding delta2
    delta2=error.mul(der)
    #using bootstrapping
    delta2=EncryptedDL1layer.bootstrapping(delta2,ctx_training)
    #finding delta1
    del1=delta2.mm(self.weight2.transpose())
    del1=EncryptedDL1layer.bootstrapping(del1,ctx_training)
    #finding der1
    der1=EncryptedDL1layer.sigmoid_derv(z1)
    #using bootstrapping
    der1=EncryptedDL1layer.bootstrapping(der1,ctx_training)
    #finding delta1
    delta1=del1.mul(der1)
    #finding the gradients
    #for weight2 and bias2
    self.dw2=a1.transpose().mm(delta2)
    self.db2=delta2.sum()
    #for weight1 and bias1
    self.dw1=enc_x_train.transpose().mm(delta1)
    self.db1=delta1.sum()
  def update_params(self):
    self.weight2=self.weight2-0.01*self.dw2
    self.bias2=self.bias2-0.01*self.db2
    self.weight1=self.weight1-0.01*self.dw1
    self.bias1=self.bias1-0.01*self.db1
  def encrypt(self, context):
    self.weight1 = ts.ckks_tensor(context, self.weight1)
    self.bias1 = ts.ckks_tensor(context, self.bias1)
    self.weight2 = ts.ckks_tensor(context, self.weight2)
    self.bias2 = ts.ckks_tensor(context, self.bias2)
  def decrypt(self):
    self.weight1 = self.weight1.decrypt()
    self.bias1 = self.bias1.decrypt()
    self.weight2 = self.weight2.decrypt()
    self.bias2 = self.bias2.decrypt()
  def accuracy(self,x_test,y_test):
    #self.decrypt()
    w1 = torch.tensor(self.weight1)
    b1 = torch.tensor(self.bias1)
    out1 = torch.sigmoid(x_test.matmul(w1) + b1).reshape(-1, 1)
    w2 = torch.tensor(self.weight2)
    b2 = torch.tensor(self.bias2)
    out2 = torch.sigmoid(out1.matmul(w2) + b2).reshape(-1, 1)
    correct = torch.abs(y_test - out2) < 0.5
    return correct.float().mean()

In [7]:
EPOCHS = 10
eelr = EncryptedDL1layer()
#print(f"enc_x_train shape: {enc_x_train.shape}")
#print(f"self.weight1 shape: {eelr.weight1.shape}")
times = []
for epoch in range(EPOCHS):
    #accuracy = eelr.accuracy(x_test, y_test)
    #print(f"Accuracy at epoch #0 is {accuracy}")
    print("Epoch : ",epoch)
    t_start = time()
    for i in range(0,780,60):
        print("Batch : ",i/60)
        t_start = time()
        enc_x_train = ts.ckks_tensor(ctx_training, x_train[i:i+60])
        enc_y_train = ts.ckks_tensor(ctx_training, y_train[i:i+60])
        t_end = time()
        print(f"Encryption of the training_batch took {int(t_end - t_start)} seconds")
        eelr.encrypt(ctx_training)
        #print(f"Is enc_x_train encrypted? {isinstance(enc_x_train, ts.CKKSTensor)}")
        #print(f"Is self.weight1 encrypted? {isinstance(eelr.weight1, ts.CKKSTensor)}")
        # if you want to keep an eye on the distribution to make sure
        # the function approximation is still working fine
        # WARNING: this operation is time consuming
        # encrypted_out_distribution(eelr, enc_x_train)
        
        #for enc_x, enc_y in zip(enc_x_train, enc_y_train):
        a2,z2,a1,z1 = eelr.forward(enc_x_train,ctx_training)
        eelr.backward(a2,z2,a1,z1,enc_y_train,ctx_training)
        print(eelr.dw1.decrypt().tolist())
        eelr.update_params()
        t_end = time()
        eelr.decrypt()
        print("Time taken for this batch is : ",t_end - t_start)
    times.append(t_end - t_start)

        #accuracy = eelr.accuracy(x_test, y_test)
        #print(f"Accuracy at epoch #{epoch + 1} is {accuracy}")
print(f"\nAverage time per epoch: {int(sum(times) / len(times))} seconds")
    #print(f"Final accuracy is {accuracy}")
print("Final weight1 ",eelr.weight1.tolist())
print("Final bias 1",eelr.bias1.tolist())
print("Final weight 2",eelr.weight2.tolist())
print("Final bias 2",eelr.bias2.tolist())

Epoch :  0
Batch :  0.0
Encryption of the training_batch took 3 seconds
[[-0.1981449980275511, -0.12852514779390833, -0.20592395301718852], [-0.13717123395606948, -0.1120549609532934, -0.15720994924059747], [-0.4287048026562168, -0.17409705819202903, -0.36686716186231166], [-0.22838739349602086, 0.00035554883650464053, -0.1638852164637052], [0.008481351601890574, -0.027617659085274388, 0.04205081636643723], [0.06257806416875979, 0.06925594236126922, 0.04020596447386919], [-0.03629242378048129, -0.016882982028142948, -0.07666961438036568], [0.04803516890893891, 0.010911147112732248, 0.0041410368531077085], [-0.22257224896828773, -0.04988060318942374, -0.18011677539416512], [-0.64652853669999, -0.3634170871221582, -0.7449607131374868]]
Time taken for this batch is :  66.41672968864441
Batch :  1.0
Encryption of the training_batch took 3 seconds
[[0.11224746102522433, 0.039392860615771075, 0.10535578152355356], [-1.2882375816207103, -0.7036809961027783, -1.063426881108851], [0.00606249566

In [8]:
print(eelr.weight1.tolist())
print(eelr.bias1.tolist())
print(eelr.weight2.tolist())
print(eelr.bias2.tolist())

[[-2.3137496947141045e+25, -1.552031831761499e+25, -1.3539412266195174e+25], [-2.7835053084563337e+25, -2.949795508368554e+25, -1.064076046793195e+25], [2.6373863779327386e+25, -1.3996514878064476e+25, -8.086055997887837e+24], [4.41758600357106e+25, -3.3932442692888445e+25, -1.8856676376122777e+25], [1.725756782616891e+25, -5.070878128721538e+25, 2.1343936032164342e+25], [3.326780221472374e+25, -3.483285053601023e+24, 1.3165502971133158e+24], [-3.3083588441512096e+25, -2.7342246707016676e+25, -2.526684318854049e+25], [-4.0778390104427316e+23, -6.018998118488763e+24, 9.861982187920736e+24], [2.6898055007977376e+24, -1.0325789839102152e+25, 5.049135530872047e+25], [6.06597457985022e+24, 2.0248447119505895e+24, 5.1985463638928524e+25]]
[1.1157922098986348e+30, -2.686733030488691e+29, 4.801319592907094e+29]
[[-8059943.069243103], [16116948.469429089], [-4428372.727606879]]
[4342351750090.89]


In [9]:
#eelr.encrypt(ctx_training)
j,k,l,r=eelr.forward(enc_x_train,ctx_training)
#enc_y_train.sub(j).decrypt().tolist()

In [10]:
out=torch.tensor(enc_y_train.sub(j).decrypt().tolist())
correct=torch.abs(out)<0.5
print(correct.float().mean())

tensor(0.)


In [11]:
enc_x_test = ts.ckks_tensor(ctx_training, x_test)
enc_y_test = ts.ckks_tensor(ctx_training, y_test)

In [12]:
m,n,j,k=eelr.forward(enc_x_test,ctx_training)
out=torch.tensor(enc_y_test.sub(m).decrypt().tolist())
correct=torch.abs(out)<0.5
print(correct.float().mean())

tensor(0.)
